In [1]:
from agential.prompting.standard.prompting import Standard
import os
from dotenv import load_dotenv
load_dotenv()

from agential.core.llm import LLM

llm = LLM("gpt-3.5-turbo", organization=os.getenv("OPENAI_ORGANIZATION"))

In [2]:
import json

with open('../data/hotpot_dev_v1_simplified.json', 'r') as f:
    data = json.load(f)

In [3]:
data[0]

{'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'answer': 'yes',
 'type': 'comparison'}

In [4]:
from agential.core.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_DIRECT
from agential.prompting.standard.prompts import STANDARD_INSTRUCTION_HOTPOTQA


method = Standard(
    llm=llm,
    benchmark="hotpotqa",
)

for inst in data[:10]:
    question, gt_answer = inst['question'], inst['answer']

    # Inference
    out = method.generate(
        question=question,
        examples=HOTPOTQA_FEWSHOT_EXAMPLES_DIRECT, 
        prompt=STANDARD_INSTRUCTION_HOTPOTQA,
        additional_keys={},
        num_retries=2,
        warming=[0.1, 0.4],
    )
    print(out.answer)
    break

    # Postprocessing

    # Evaluation
    # precision(out.answer, gt_answer)

[['No', 'No'], ['No', 'No']]


In [ ]:
out.additional_info[0][0]